In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import re
from matplotlib import pyplot as plt
import seaborn as sns
import glob
import re
from scipy.interpolate import interp1d
from tqdm import tqdm, tqdm_notebook
from scipy import spatial
from sklearn.linear_model import LogisticRegression

In [5]:
def calculate_pad(brightness, saturation):
    p = 0.69*brightness + 0.22*saturation
    a = -0.31*brightness + 0.6*saturation
    d = 0.76*brightness + 0.32*saturation
    return [p,a,d]

def calculate_pad_scene(scene):
    pads = []
    for img in scene:
        temp_b = mean_brightness(img)
        temp_s = mean_saturation(img)
        pads.append(calculate_pad(temp_b, temp_s))
    return np.mean([x[0] for x in pads]), np.mean([x[1] for x in pads]), np.mean([x[2] for x in pads])

def calculate_blur(img):
    return cv2.Laplacian(img, cv2.CV_64F).var()

def calculate_blur_scene(scene):
    blurs = []
    for img in scene:
        blurs.append(calculate_blur(img))
    return np.mean(blurs)

def mean_brightness(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) #convert it to hsv
    return np.mean(hsv[:,:,2])

def mean_saturation(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) #convert it to hsv
    return np.mean(hsv[:,:,1])

def calculate_opticalFlow(img1, img2):
    prev = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    forward = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    mask = np.zeros_like(prev)
    mask[..., 1] = 255
    flow = cv2.calcOpticalFlowFarneback(prev, forward, flow=None, pyr_scale=0.5,levels =1,winsize=3,iterations=15,poly_n=3,poly_sigma=5,flags=cv2.OPTFLOW_FARNEBACK_GAUSSIAN)
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    return cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)[0]

def calculate_opticalFlow_scene(scene):
    first = calculate_opticalFlow(scene[0], scene[1])
    second = calculate_opticalFlow(scene[1], scene[2])
    return np.mean([first, second])
    
def isjpg(filepath):
    return re.search(".jpg$", filepath)

def calculate_features(img_path):
    img = cv2.imread(img_path)
    b = mean_brightness(img)
    s = mean_saturation(img)
    result = calculate_pad(b, s)
    result.append(calculate_blur(img))
    return result

In [14]:
base = 'data\\outside\\training'
dirs = os.listdir(base)
labels = []
train_ps = []
train_as = []
train_ds = []
train_blurs = []
for emotion in dirs:
    emotion_path = os.path.join(base, emotion)
    for img in tqdm_notebook(os.listdir(emotion_path)):
        img_path = os.path.join(emotion_path, img)
        res = calculate_features(img_path)
        labels.append(emotion)
        train_ps.append(res[0])
        train_as.append(res[1])
        train_ds.append(res[2])
        train_blurs.append(res[3])

In [15]:
df = pd.DataFrame()
df['label'] = labels
df['valence'] = train_ps
df['arousal'] = train_as
df['dominance'] = train_ds
df['blur'] = train_blurs

In [16]:
X = df.drop[columns='label']
Y = df.drop[]

,label,valence,arousal,dominance,blur
0,anger,49.400915,37.531569,61.051597,67.234052
1,anger,109.097942,40.835449,130.153375,245.933347
2,anger,124.918752,2.291057,144.084802,94.108565
3,anger,94.230517,25.941924,111.379657,7.088119
4,anger,132.601645,46.511766,157.846255,52.640256
5,anger,105.496884,7.416722,122.292423,160.730490
6,anger,91.935737,-29.360208,102.590241,133.660629
7,anger,80.824084,-13.858200,91.519560,89.011121
8,anger,76.894430,29.580339,91.823504,110.387618
9,anger,112.047233,31.467238,132.507814,22.817304
